Run this file in Colab: https://colab.research.google.com/drive/1ppgHiXKXeTmAevLh5ewrcoDsXU_D9UDA?usp=sharing

# Data Transformation

We need our data to look something like this for py-irt

```json
{"subject_id": "pedro",    "responses": {"q1": 1, "q2": 0, "q3": 1, "q4": 0}}
{"subject_id": "pinguino", "responses": {"q1": 1, "q2": 1, "q3": 0, "q4": 0}}
{"subject_id": "ken",      "responses": {"q1": 1, "q2": 1, "q3": 1, "q4": 1}}
{"subject_id": "burt",     "responses": {"q1": 0, "q2": 0, "q3": 0, "q4": 0}}
```


## Setup

```
conda create -n ENVNAME python=3.10 pandas
conda activate ENVNAME
```

```
conda install -c conda-forge jupyterlab
pip install pandas pyro-ppl py-irt
```

In [3]:
!pip install pandas pyro-ppl py-irt


Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pandas as pd
import json

In [5]:
import urllib.request
urllib.request.urlretrieve("https://www.johnlalor.net/cola.rp", "cola.rp")

('cola.rp', <http.client.HTTPMessage at 0x70a511aa0df0>)

In [6]:
D = pd.read_csv("cola.rp", sep="\t", names=["modelID","itemID","response"])

D.head()

,modelID,itemID,response
0,m-0.34,0,1
1,m-0.34,1,1
2,m-0.34,2,1
3,m-0.34,3,1
4,m-0.34,4,1


In [7]:
D.head(-10)

,modelID,itemID,response
0,m-0.34,0,1
1,m-0.34,1,1
2,m-0.34,2,1
3,m-0.34,3,1
4,m-0.34,4,1
...,...,...,...
427535,m-0.04,8536,1
427536,m-0.04,8537,1
427537,m-0.04,8538,1
427538,m-0.04,8539,0


In [8]:
len(D['modelID'].unique())

50

In [9]:
response_patterns = {}

for idx, row in D.iterrows():
    if row["modelID"] not in response_patterns:
        response_patterns[row["modelID"]] = {}
    response_patterns[row["modelID"]][f"q{row['itemID']}"] = row["response"]

In [10]:
with open("cola_pyirt.jsonlines", "w") as outfile:
    for key, val in response_patterns.items():
        outrow = {"subject_id": key, "responses": val}
        outfile.write(json.dumps(outrow) + "\n")

*[jump to after results](#afterResults)

In [11]:
!py-irt train 1pl cola_pyirt.jsonlines output/cola1pl

[22:54:38] config: model_type='1pl' epochs=2000 priors=None           ]8;id=891540;file:///home/lalor/.local/lib/python3.10/site-packages/py_irt/cli.py\cli.py]8;;\:]8;id=758169;file:///home/lalor/.local/lib/python3.10/site-packages/py_irt/cli.py#109\109]8;;\
           initializers=[] dims=None lr=0.1 lr_decay=0.9999                     
           dropout=0.5 hidden=100 vocab_size=None log_every=100                 
           seed=None deterministic=False                                        
           data_path: cola_pyirt.jsonlines                            ]8;id=946112;file:///home/lalor/.local/lib/python3.10/site-packages/py_irt/cli.py\cli.py]8;;\:]8;id=479025;file:///home/lalor/.local/lib/python3.10/site-packages/py_irt/cli.py#111\111]8;;\
           output directory: output/cola1pl                           ]8;id=795814;file:///home/lalor/.local/lib/python3.10/site-packages/py_irt/cli.py\cli.py]8;;\:]8;id=864928;file:///home/lalor/.local/lib/python3.10/

<a id="afterResults"></a>

To view the output results, use either

```
!cat output/cola1pl/best_parameters.json
```

or

```
!type output\cola1pl\best_parameters.json
```

depending on your operating system.

In [12]:
with open ("output/cola1pl/best_parameters.json","r") as file:
    data = json.load(file)
data.keys()

dict_keys(['ability', 'diff', 'irt_model', 'item_ids', 'subject_ids'])

In [13]:
len(data["diff"])

8551

In [14]:
items = pd.DataFrame(
    index=data["item_ids"], 
    data= data["diff"],
    columns=["diff"]
)
items.head()

,diff
0,-22.281200
1,-21.512703
2,-19.260849
3,-19.801846
4,-18.036003


In [15]:
subjects = pd.DataFrame(
    index=data["subject_ids"], 
    data= data["ability"],
    columns=["ability"]
)
subjects.head()

,ability
0,9.439343
1,10.120603
2,8.916849
3,9.229851
4,8.850241
